# HealthBot: AI-Powered Medical Information Agent
## Interactive Workflow for Health Education

This notebook demonstrates a complete AI agent system that:
1. Takes patient health topics
2. Searches medical information via Tavily
3. Summarizes findings at an 8th-grade reading level
4. Generates quiz questions to assess understanding
5. Grades answers with detailed feedback

**Stand-out Feature**: For each health topic, the system generates NEW quiz questions as you practice, enabling deeper learning.

In [ ]:
import os
import sys
from pathlib import Path
from typing import TypedDict, Annotated, Optional, List
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

print('All imports successful')

In [ ]:
project_root = Path(r'c:\Training\Udacity\AI_Agents_LangGraph\project')
env_path = project_root / '.env'

load_dotenv(env_path, override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')

if not openai_api_key:
    raise ValueError('OPENAI_API_KEY not found')
if not tavily_api_key:
    raise ValueError('TAVILY_API_KEY not found')

print(f'Env loaded from: {env_path}')
print(f'OpenAI key: {bool(openai_api_key)}')
print(f'Tavily key: {bool(tavily_api_key)}')

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)

print(f'LLM: {llm.model_name}')
print(f'Temperature: {llm.temperature}')

In [ ]:
tavily_tool = TavilySearchResults(max_results=5, api_key=tavily_api_key)

print(f'Tavily tool initialized')
print(f'Max results: {tavily_tool.max_results}')

In [ ]:
class HealthBotState(TypedDict):
    messages: Annotated[List, add_messages]
    health_topic: Optional[str]
    search_results: Optional[str]
    summary: Optional[str]
    quiz_question: Optional[str]
    patient_answer: Optional[str]
    grade: Optional[str]
    feedback: Optional[str]
    should_continue: str
    quiz_count: int

print(f'State schema defined with {len(HealthBotState.__annotations__)} fields')

In [ ]:
# Import required libraries
import os
import sys
from pathlib import Path
from typing import TypedDict, Annotated, Optional, List
from dotenv import load_dotenv

# LangChain and LangGraph
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

print('All imports successful')

In [ ]:
# Setup project paths
project_root = Path(r'c:\Training\Udacity\AI_Agents_LangGraph\project')
env_path = project_root / '.env'

# Load environment variables
load_dotenv(env_path, override=True)

# Verify credentials are loaded
openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')

if not openai_api_key:
    raise ValueError('OPENAI_API_KEY not found in .env')
if not tavily_api_key:
    raise ValueError('TAVILY_API_KEY not found in .env')

print(f'Project root: {project_root}')
print(f'OpenAI API key loaded: {bool(openai_api_key)}')
print(f'Tavily API key loaded: {bool(tavily_api_key)}')

## Step 1: Initialize LLM

Create the OpenAI language model instance.

In [ ]:
# Initialize OpenAI LLM
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)

print('LLM initialized: gpt-3.5-turbo')
print(f'Model: {llm.model_name}')
print(f'Temperature: {llm.temperature}')

## Step 2: Initialize Tavily Search Tool

Create the medical search tool.

In [ ]:
# Initialize Tavily search tool
tavily_tool = TavilySearchResults(
    max_results=5,
    api_key=tavily_api_key
)

print('Tavily search tool initialized')
print(f'Max results per search: {tavily_tool.max_results}')

## Step 3: Define State Schema

Define all state fields that flow through the agent.

In [ ]:
class HealthBotState(TypedDict):
    '''State schema for HealthBot workflow'''
    messages: Annotated[List, add_messages]  # Conversation history
    health_topic: Optional[str]              # Patient's health topic
    search_results: Optional[str]            # Raw Tavily search results
    summary: Optional[str]                   # LLM-generated summary
    quiz_question: Optional[str]             # Generated quiz question
    patient_answer: Optional[str]            # Patient's answer to quiz
    grade: Optional[str]                     # Letter grade (A-F)
    feedback: Optional[str]                  # Grading feedback with citations
    should_continue: str                     # 'start', 'continue', or 'exit'
    quiz_count: int                          # Number of quiz questions asked

print('State schema defined:')
print(f'Fields: {list(HealthBotState.__annotations__.keys())}')

## Setup: Environment and Credentials

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Project paths
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Load credentials from .env
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

# Mark that env is loaded so modules skip redundant loading
os.environ['ENV_ALREADY_LOADED'] = 'true'

# Add src to Python path
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

print(f"✓ Project root: {project_root}")
print(f"✓ Credentials loaded: OpenAI={bool(openai_key)}, Tavily={bool(tavily_key)}")
print(f"✓ Ready to run HealthBot")

✓ Project root: c:\Training\Udacity\AI_Agents_LangGraph\project
✓ Credentials loaded: OpenAI=True, Tavily=True
✓ Ready to run HealthBot


## Import and Initialize Workflow

In [2]:
# Import workflow components
from workflow import create_healthbot_workflow

# Create the compiled workflow
app = create_healthbot_workflow()

print("✓ Workflow imported and compiled")
print("✓ Ready for interactive session")

✓ Workflow imported and compiled
✓ Ready for interactive session


## Run Interactive HealthBot Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

# Configuration with thread ID for checkpointing
config = {"configurable": {"thread_id": "notebook-session-1"}}

print("="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)
print()

# Run the workflow
try:
    result = app.invoke(initial_state, config)
    
    # Display results
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Topic: {result.get('health_topic', 'N/A')}")
    print(f"Grade: {result.get('grade', 'N/A')}/100")
    print(f"Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture: 8-Node AI Agent Workflow

### Core Workflow Nodes:

1. **ask_for_topic** - Interactive input: Collects patient's health topic
2. **search_medical_info** - Tavily API: Searches for credible medical sources
3. **summarize_results** - LLM processing: Creates 8th-grade-level summary
4. **present_summary** - Display: Shows summary and sources
5. **generate_quiz** - LLM processing: Generates NEW quiz question each time
6. **present_quiz** - Display: Shows question with 4 multiple choice options
7. **evaluate_answer** - LLM processing: Grades answer with detailed feedback
8. **ask_continue** - Conditional Routing (3-way junction):
   - "q" → Route back to Node 5 (new question on same topic)
   - "t" → Route to Node 1 (new topic)
   - "e" → End session

### Stand-Out Feature: Dynamic Question Generation

**Key Differentiator**: Each quiz question is AI-generated in real-time, NOT pre-created:
- Node 5 explicitly instructs the LLM to create a NEW question never before asked
- Same topic can produce unlimited unique questions
- Enables **comprehensive topic mastery** through multiple angles
- Prevents **rote memorization** - students must deeply understand
- Multiple **learning reinforcement checkpoints**

### Technical Implementation:
- **Framework**: LangGraph 0.2.19 (graph-based state machine)
- **Language Model**: OpenAI GPT-3.5-turbo
- **Search Engine**: Tavily API (medical information retrieval)
- **State Management**: 10-field state with thread-based checkpointing
- **Execution Model**: Synchronous invoke with user interaction via input()

### Data Flow:
```
User Input → Topic Search → LLM Summary → Display → 
LLM Generate Question → Display Question → User Answer → 
LLM Evaluate → Display Grade → Router Decision → 
(More Q / New Topic / Exit)
```

## Setup: Environment and Credentials

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Project paths
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Load credentials from .env
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

# Mark that env is loaded so modules skip redundant loading
os.environ['ENV_ALREADY_LOADED'] = 'true'

# Add src to Python path
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

print(f"✓ Project root: {project_root}")
print(f"✓ Credentials loaded: OpenAI={bool(openai_key)}, Tavily={bool(tavily_key)}")
print(f"✓ Ready to run HealthBot")

## Import and Initialize Workflow

In [ ]:
# Import workflow components
from workflow import create_healthbot_workflow

# Create the compiled workflow
app = create_healthbot_workflow()

print("✓ Workflow imported and compiled")
print("✓ Ready for interactive session")

## Run Interactive HealthBot Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

# Configuration with thread ID for checkpointing
config = {"configurable": {"thread_id": "notebook-session-1"}}

print("="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)
print()

# Run the workflow
try:
    result = app.invoke(initial_state, config)
    
    # Display results
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Topic: {result.get('health_topic', 'N/A')}")
    print(f"Grade: {result.get('grade', 'N/A')}/100")
    print(f"Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture Overview: 8-Node AI Agent Workflow

### Core Workflow Nodes:

1. **ask_for_topic** - Interactive input: Collects patient's health topic
2. **search_medical_info** - Tavily API: Searches for credible medical sources
3. **summarize_results** - LLM processing: Creates 8th-grade-level summary
4. **present_summary** - Display: Shows summary and sources
5. **generate_quiz** - LLM processing: Generates NEW quiz question each time
6. **present_quiz** - Display: Shows question with 4 multiple choice options
7. **evaluate_answer** - LLM processing: Grades answer with detailed feedback
8. **ask_continue** - Conditional Routing (3-way junction):
   - "q" → Route back to Node 5 (new question on same topic)
   - "t" → Route to Node 1 (new topic)
   - "e" → End session

### Stand-Out Feature: Dynamic Question Generation

**Key Differentiator**: Each quiz question is AI-generated in real-time, NOT pre-created:
- Node 5 explicitly instructs the LLM to create a NEW question never before asked
- Same topic can produce unlimited unique questions
- Enables **comprehensive topic mastery** through multiple angles
- Prevents **rote memorization** - students must deeply understand
- Multiple **learning reinforcement checkpoints**

### Technical Implementation:
- **Framework**: LangGraph 0.2.19 (graph-based state machine)
- **Language Model**: OpenAI GPT-3.5-turbo
- **Search Engine**: Tavily API (medical information retrieval)
- **State Management**: 10-field state with thread-based checkpointing
- **Execution Model**: Synchronous invoke with user interaction via input()

### Data Flow:
```
User Input → Topic Search → LLM Summary → Display → 
LLM Generate Question → Display Question → User Answer → 
LLM Evaluate → Display Grade → Router Decision → 
(More Q / New Topic / Exit)
```

## Setup: Environment & Credentials

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Project paths
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Load credentials from .env
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

# Mark that env is loaded so modules skip redundant loading
os.environ['ENV_ALREADY_LOADED'] = 'true'

# Add src to Python path
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

print(f"✓ Project root: {project_root}")
print(f"✓ Credentials loaded: OpenAI={bool(openai_key)}, Tavily={bool(tavily_key)}")
print(f"✓ Ready to run HealthBot")

## Import & Initialize Workflow

In [ ]:
# Import workflow components
from workflow import create_healthbot_workflow

# Create the compiled workflow
app = create_healthbot_workflow()

print("✓ Workflow imported and compiled")
print("✓ Ready for interactive session")

## Run Interactive HealthBot Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

# Configuration with thread ID for checkpointing
config = {"configurable": {"thread_id": "notebook-session-1"}}

print("="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)
print()

# Run the workflow
try:
    result = app.invoke(initial_state, config)
    
    # Display results
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Topic: {result.get('health_topic', 'N/A')}")
    print(f"Grade: {result.get('grade', 'N/A')}/100")
    print(f"Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture: 8-Node Workflow with Dynamic Questions

### Workflow Nodes:

1. **ask_for_topic** - Gathers patient's health question
2. **search_medical_info** - Tavily search for credible sources
3. **summarize_results** - LLM summarization at 8th-grade level
4. **present_summary** - Display summary and sources
5. **generate_quiz** - LLM generates NEW quiz question (different each time)
6. **present_quiz** - Display question with 4 options
7. **evaluate_answer** - LLM grades with feedback
8. **ask_continue** - 3-way routing:
   - More questions → Generate new question on same topic (Node 5)
   - New topic → Start over (Node 1)
   - Exit → End session

### Stand-Out Feature: Dynamic Question Generation

Unlike traditional systems that repeat the same question, HealthBot generates **different questions for each attempt**:
- Node 5 explicitly instructs the LLM to create a NEW question not previously asked
- Enables **deeper learning** through multiple angles
- Prevents **memorization** rather than understanding
- Multiple **checkpoints for comprehension**

### Tech Stack:
- **LangGraph 0.2.19** - Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI** - Language model
- **Tavily Search API** - Medical information retrieval
- **Python 3.8+** - Implementation

## Setup: Environment & Credentials

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Project paths
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Load credentials from .env
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

# Mark that env is loaded so modules skip redundant loading
os.environ['ENV_ALREADY_LOADED'] = 'true'

# Add src to Python path
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

print(f"✓ Project root: {project_root}")
print(f"✓ Credentials loaded: OpenAI={bool(openai_key)}, Tavily={bool(tavily_key)}")
print(f"✓ Ready to run HealthBot")

## Import & Initialize Workflow

In [ ]:
# Import workflow components
from workflow import create_healthbot_workflow

# Create the compiled workflow
app = create_healthbot_workflow()

print("✓ Workflow imported and compiled")
print("✓ Ready for interactive session")

## Run Interactive HealthBot Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

# Configuration with thread ID for checkpointing
config = {"configurable": {"thread_id": "notebook-session-1"}}

print("="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)
print()

# Run the workflow
try:
    result = app.invoke(initial_state, config)
    
    # Display results
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Topic: {result.get('health_topic', 'N/A')}")
    print(f"Grade: {result.get('grade', 'N/A')}/100")
    print(f"Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture: 8-Node Workflow with Dynamic Questions

### Workflow Nodes:

1. **ask_for_topic** - Gathers patient's health question
2. **search_medical_info** - Tavily search for credible sources
3. **summarize_results** - LLM summarization at 8th-grade level
4. **present_summary** - Display summary and sources
5. **generate_quiz** - LLM generates NEW quiz question (different each time)
6. **present_quiz** - Display question with 4 options
7. **evaluate_answer** - LLM grades with feedback
8. **ask_continue** - 3-way routing:
   - More questions → Generate new question on same topic (Node 5)
   - New topic → Start over (Node 1)
   - Exit → End session

### Stand-Out Feature: Dynamic Question Generation

Unlike traditional systems that repeat the same question, HealthBot generates **different questions for each attempt**:
- Node 5 explicitly instructs the LLM to create a NEW question not previously asked
- Enables **deeper learning** through multiple angles
- Prevents **memorization** rather than understanding
- Multiple **checkpoints for comprehension**

### Tech Stack:
- **LangGraph 0.2.19** - Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI** - Language model
- **Tavily Search API** - Medical information retrieval
- **Python 3.8+** - Implementation

## Setup: Environment & Path Configuration

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Use absolute path to project root where .env is located
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Load .env IMMEDIATELY so it's available for module initialization
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

# Mark that env is already loaded - this tells modules to skip loading
os.environ['ENV_ALREADY_LOADED'] = 'true'

# Add src to path and change working directory
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

print(f"✓ Project root: {project_root}")
print(f"✓ Src path: {src_path}")
print(f"✓ Working directory: {os.getcwd()}")
print(f"✓ ENV_ALREADY_LOADED marker set")

## Load Environment & Verify Credentials

In [ ]:
# Verify credentials (already loaded in cell 3)
openai_key = os.getenv('OPENAI_API_KEY', '').strip()
tavily_key = os.getenv('TAVILY_API_KEY', '').strip()
foundry_endpoint = os.getenv('FOUNDRY_PROJECT_ENDPOINT', '').strip()

env_path = project_root / '.env'
print(f"✓ .env file: {env_path}")
print(f"✓ File exists: {env_path.exists()}")
print(f"✓ OpenAI API Key loaded: {'Yes' if openai_key else 'No'}")
print(f"✓ Tavily API Key loaded: {'Yes' if tavily_key else 'No'}")
print(f"✓ Foundry Endpoint loaded: {'Yes' if foundry_endpoint else 'No'}")

if not tavily_key:
    raise ValueError("Missing TAVILY_API_KEY in .env")
print("✓ All required credentials verified!")

## Import HealthBot Modules

In [ ]:
# Import modules
from llm_config import initialize_llm
from workflow import create_healthbot_workflow

print("✓ llm_config imported")
print("✓ workflow imported")
print("✓ All modules loaded successfully!")

## Initialize LLM & Create Workflow

In [ ]:
# Initialize LLM directly (credentials already in environment from cell 5)
from langchain_openai import ChatOpenAI

# Use OpenAI with credentials already loaded in environment
llm = ChatOpenAI(model="gpt-3.5-turbo")
print(f"✓ LLM initialized: {llm.__class__.__name__}")

# Create workflow
from workflow import create_healthbot_workflow
app = create_healthbot_workflow()
print("✓ Workflow created")
print(f"✓ Workflow compiled and ready to run")

## Initialize State & Run Interactive Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

print("✓ Initial state prepared")
print("\n" + "="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)

In [ ]:
# Run the HealthBot CLI script which handles environment loading properly
import subprocess
import sys

cli_script = project_root / "healthbot" / "run_healthbot.py"

print(f"Running: {cli_script}\n")

try:
    result = subprocess.run(
        [sys.executable, str(cli_script)],
        cwd=str(project_root),
        capture_output=False,
        text=True
    )
    
    if result.returncode == 0:
        print("\n✓ Session completed successfully!")
    else:
        print(f"\n❌ CLI script exited with code {result.returncode}")
        
except Exception as e:
    print(f"❌ Error running CLI script: {e}")
    import traceback
    traceback.print_exc()

## Architecture Overview: 8-Node Workflow with Dynamic Question Generation

### Workflow Nodes:

1. **ask_for_topic**: Gathers patient's health question with validation
2. **search_medical_info**: Uses Tavily API to find credible medical sources (5 sources)
3. **summarize_results**: LLM summarizes at 8th-grade reading level
4. **present_summary**: Displays summary and sources to patient
5. **generate_quiz**: LLM generates quiz question (different for each attempt)
6. **present_quiz**: Displays question with 4 multiple-choice options
7. **evaluate_answer**: LLM grades answer and provides feedback with citations
8. **ask_continue**: 3-way routing:
   - "1" → More questions (loops to Node 5 with new question)
   - "2" → New topic (loops to Node 1)
   - "3" → Exit session

### Stand-Out Feature: Dynamic Question Generation

**Problem**: Traditional quiz systems ask the same question repeatedly, leading to memorization rather than learning.

**Solution**: HealthBot generates **different questions for each attempt** on the same topic. Node 5 checks `quiz_count` and explicitly instructs LLM: "Generate a NEW question, not the one previously asked."

**Benefits**:
- **Deeper learning**: Multiple angles on same topic
- **Better retention**: Prevents memorization
- **Comprehensive assessment**: Multiple checkpoints for understanding

### Technology Stack:
- **LangGraph 0.2.19**: Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI**: Language model for summarization and grading  
- **Tavily Search API**: Medical information retrieval with credible sources
- **Python 3.8+**: Implementation language

### Message Flow:
```
Patient Input
    ↓
Search Tavily → Get 5 Sources
    ↓
Summarize → 8th Grade Level
    ↓
Present Summary
    ↓
Generate NEW Quiz Question (different each time)
    ↓
Get Patient Answer
    ↓
Grade with LLM → Feedback + Citations
    ↓
Continue? (More Questions | New Topic | Exit)
    ↓
Loop back or End Session
```

## Setup: Environment & Path Configuration

In [ ]:
import os
import sys
from pathlib import Path

# The .env is at: c:\Training\Udacity\AI_Agents_LangGraph\project\.env
# Use absolute path to be sure it works
project_root = Path(r"c:\Training\Udacity\AI_Agents_LangGraph\project")
src_path = project_root / "healthbot" / "src"

# Add src to path and change working directory
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

print(f"✓ Project root: {project_root}")
print(f"✓ Src path added: {src_path}")
print(f"✓ Working directory: {os.getcwd()}")

## Load Environment & Verify Credentials

In [ ]:
from dotenv import load_dotenv

# Load .env from project root
env_path = project_root / '.env'
load_dotenv(env_path)

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY', '').strip()
tavily_key = os.getenv('TAVILY_API_KEY', '').strip()

print(f"✓ .env file: {env_path}")
print(f"✓ File exists: {env_path.exists()}")
print(f"✓ OpenAI API Key loaded: {'Yes' if openai_key else 'No'}")
print(f"✓ Tavily API Key loaded: {'Yes' if tavily_key else 'No'}")

if not openai_key or not tavily_key:
    raise ValueError("Missing required credentials in .env")
print("✓ All credentials verified!")

## Import HealthBot Modules

In [ ]:
# Import modules
from llm_config import initialize_llm
from workflow import create_healthbot_workflow

print("✓ llm_config imported")
print("✓ workflow imported")
print("✓ All modules loaded successfully!")

## Initialize LLM & Create Workflow

In [ ]:
# Initialize LLM
llm = initialize_llm()
print(f"✓ LLM initialized: {llm.__class__.__name__}")

# Create workflow
app = create_healthbot_workflow()
print("✓ Workflow created")
print(f"✓ Workflow compiled and ready to run")

## Initialize State & Run Interactive Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

print("✓ Initial state prepared")
print("\n" + "="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)

In [ ]:
# Run the workflow interactively
try:
    result = app.invoke(initial_state)
    
    # Display final session summary
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Health Topic: {result.get('health_topic', 'N/A')}")
    print(f"Final Grade: {result.get('grade', 'N/A')}/100")
    print(f"Quiz Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages Exchanged: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture Overview: 8-Node Workflow with Dynamic Question Generation

### Workflow Nodes:

1. **ask_for_topic**: Gathers patient's health question with validation
2. **search_medical_info**: Uses Tavily API to find credible medical sources (5 sources)
3. **summarize_results**: LLM summarizes at 8th-grade reading level
4. **present_summary**: Displays summary and sources to patient
5. **generate_quiz**: LLM generates quiz question (different for each attempt)
6. **present_quiz**: Displays question with 4 multiple-choice options
7. **evaluate_answer**: LLM grades answer and provides feedback with citations
8. **ask_continue**: 3-way routing:
   - "1" → More questions (loops to Node 5 with new question)
   - "2" → New topic (loops to Node 1)
   - "3" → Exit session

### Stand-Out Feature: Dynamic Question Generation

**Problem**: Traditional quiz systems ask the same question repeatedly, leading to memorization rather than learning.

**Solution**: HealthBot generates **different questions for each attempt** on the same topic. Node 5 checks `quiz_count` and explicitly instructs LLM: "Generate a NEW question, not the one previously asked."

**Benefits**:
- **Deeper learning**: Multiple angles on same topic
- **Better retention**: Prevents memorization
- **Comprehensive assessment**: Multiple checkpoints for understanding

### Technology Stack:
- **LangGraph 0.2.19**: Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI**: Language model for summarization and grading  
- **Tavily Search API**: Medical information retrieval with credible sources
- **Python 3.8+**: Implementation language

### Message Flow:
```
Patient Input
    ↓
Search Tavily → Get 5 Sources
    ↓
Summarize → 8th Grade Level
    ↓
Present Summary
    ↓
Generate NEW Quiz Question (different each time)
    ↓
Get Patient Answer
    ↓
Grade with LLM → Feedback + Citations
    ↓
Continue? (More Questions | New Topic | Exit)
    ↓
Loop back or End Session
```

## Setup: Environment & Path Configuration

In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (.env location)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent  # notebooks -> healthbot -> project
src_path = project_root / 'healthbot' / 'src'

# Add src to path for module imports
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

print(f"✓ Project root: {project_root}")
print(f"✓ Src path added: {src_path}")
print(f"✓ Working directory: {os.getcwd()}")

## Load Environment & Verify Credentials

In [ ]:
from dotenv import load_dotenv

# Load .env from project root
env_path = project_root / '.env'
load_dotenv(env_path)

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY', '').strip()
tavily_key = os.getenv('TAVILY_API_KEY', '').strip()

print(f"✓ .env file: {env_path}")
print(f"✓ File exists: {env_path.exists()}")
print(f"✓ OpenAI API Key loaded: {'Yes' if openai_key else 'No'}")
print(f"✓ Tavily API Key loaded: {'Yes' if tavily_key else 'No'}")

if not openai_key or not tavily_key:
    raise ValueError("Missing required credentials in .env")
print("✓ All credentials verified!")

## Import HealthBot Modules

In [ ]:
# Import modules
from llm_config import initialize_llm
from state import HealthBotState
from workflow import create_healthbot_workflow

print("✓ llm_config imported")
print("✓ state imported")
print("✓ workflow imported")
print("✓ All modules loaded successfully!")

## Initialize LLM & Create Workflow

In [ ]:
# Initialize LLM
llm = initialize_llm()
print(f"✓ LLM initialized: {llm.__class__.__name__}")

# Create workflow
app = create_healthbot_workflow()
print("✓ Workflow created")
print(f"✓ Workflow compiled and ready to run")

## Initialize State & Run Interactive Session

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

print("✓ Initial state prepared")
print("\n" + "="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)

In [ ]:
# Run the workflow interactively
try:
    result = app.invoke(initial_state)
    
    # Display final session summary
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Health Topic: {result.get('health_topic', 'N/A')}")
    print(f"Final Grade: {result.get('grade', 'N/A')}/100")
    print(f"Quiz Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages Exchanged: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Architecture Overview: 8-Node Workflow with Dynamic Question Generation

### Workflow Nodes:

1. **ask_for_topic**: Gathers patient's health question with validation
2. **search_medical_info**: Uses Tavily API to find credible medical sources (5 sources)
3. **summarize_results**: LLM summarizes at 8th-grade reading level
4. **present_summary**: Displays summary and sources to patient
5. **generate_quiz**: LLM generates quiz question (different for each attempt)
6. **present_quiz**: Displays question with 4 multiple-choice options
7. **evaluate_answer**: LLM grades answer and provides feedback with citations
8. **ask_continue**: 3-way routing:
   - "1" → More questions (loops to Node 5 with new question)
   - "2" → New topic (loops to Node 1)
   - "3" → Exit session

### Stand-Out Feature: Dynamic Question Generation

**Problem**: Traditional quiz systems ask the same question repeatedly, leading to memorization rather than learning.

**Solution**: HealthBot generates **different questions for each attempt** on the same topic. Node 5 checks `quiz_count` and explicitly instructs LLM: "Generate a NEW question, not the one previously asked."

**Benefits**:
- **Deeper learning**: Multiple angles on same topic
- **Better retention**: Prevents memorization
- **Comprehensive assessment**: Multiple checkpoints for understanding

### Technology Stack:
- **LangGraph 0.2.19**: Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI**: Language model for summarization and grading  
- **Tavily Search API**: Medical information retrieval with credible sources
- **Python 3.8+**: Implementation language

### Message Flow:
```
Patient Input
    ↓
Search Tavily → Get 5 Sources
    ↓
Summarize → 8th Grade Level
    ↓
Present Summary
    ↓
Generate NEW Quiz Question (different each time)
    ↓
Get Patient Answer
    ↓
Grade with LLM → Feedback + Citations
    ↓
Continue? (More Questions | New Topic | Exit)
    ↓
Loop back or End Session
```

## Architecture Overview: 8-Node Workflow with Dynamic Question Generation

### Workflow Nodes:

1. **ask_for_topic**: Gathers patient's health question with validation
2. **search_medical_info**: Uses Tavily API to find credible medical sources (5 sources)
3. **summarize_results**: LLM summarizes at 8th-grade reading level
4. **present_summary**: Displays summary and sources to patient
5. **generate_quiz**: LLM generates quiz question (different for each attempt)
6. **present_quiz**: Displays question with 4 multiple-choice options
7. **evaluate_answer**: LLM grades answer and provides feedback with citations
8. **ask_continue**: 3-way routing:
   - "1" → More questions (loops to Node 5 with new question)
   - "2" → New topic (loops to Node 1)
   - "3" → Exit session

### Stand-Out Feature: Dynamic Question Generation

**Problem**: Traditional quiz systems ask the same question repeatedly, leading to memorization rather than learning.

**Solution**: HealthBot generates **different questions for each attempt** on the same topic. Node 5 checks `quiz_count` and explicitly instructs LLM: "Generate a NEW question, not the one previously asked."

**Benefits**:
- **Deeper learning**: Multiple angles on same topic
- **Better retention**: Prevents memorization
- **Comprehensive assessment**: Multiple checkpoints for understanding

### Technology Stack:
- **LangGraph 0.2.19**: Workflow orchestration with conditional routing
- **OpenAI ChatOpenAI**: Language model for summarization and grading  
- **Tavily Search API**: Medical information retrieval with credible sources
- **Python 3.8+**: Implementation language

### Message Flow:
```
Patient Input
    ↓
Search Tavily → Get 5 Sources
    ↓
Summarize → 8th Grade Level
    ↓
Present Summary
    ↓
Generate NEW Quiz Question (different each time)
    ↓
Get Patient Answer
    ↓
Grade with LLM → Feedback + Citations
    ↓
Continue? (More Questions | New Topic | Exit)
    ↓
Loop back or End Session
```

In [ ]:
# Run the workflow interactively
try:
    result = app.invoke(initial_state)
    
    # Display final session summary
    print("\n" + "="*70)
    print("SESSION SUMMARY")
    print("="*70)
    print(f"Health Topic: {result.get('health_topic', 'N/A')}")
    print(f"Final Grade: {result.get('grade', 'N/A')}/100")
    print(f"Quiz Questions Asked: {result.get('quiz_count', 0)}")
    print(f"Messages Exchanged: {len(result.get('messages', []))}")
    print("="*70)
    print("✓ Session completed successfully!")
    
except KeyboardInterrupt:
    print("\n✓ Session ended by user")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Initialize state
initial_state = {
    "messages": [],
    "health_topic": None,
    "search_results": None,
    "summary": None,
    "quiz_question": None,
    "patient_answer": None,
    "grade": None,
    "feedback": None,
    "should_continue": "start",
    "session_id": "notebook-session",
    "quiz_count": 0
}

print("✓ Initial state prepared")
print("\n" + "="*70)
print("HEALTHBOT INTERACTIVE SESSION")
print("="*70)

## Initialize State & Run Interactive Session

In [ ]:
# Initialize LLM
llm = initialize_llm()
print(f"✓ LLM initialized: {llm.__class__.__name__}")

# Create workflow
app = create_healthbot_workflow()
print("✓ Workflow created")
print(f"✓ Workflow compiled and ready to run")

## Initialize LLM & Create Workflow

In [ ]:
# Import modules
from llm_config import initialize_llm
from state import HealthBotState
from workflow import create_healthbot_workflow

print("✓ llm_config imported")
print("✓ state imported")
print("✓ workflow imported")
print("✓ All modules loaded successfully!")

## Import HealthBot Modules

In [ ]:
from dotenv import load_dotenv

# Load .env from project root
env_path = project_root / '.env'
load_dotenv(env_path)

# Verify credentials
openai_key = os.getenv('OPENAI_API_KEY', '').strip()
tavily_key = os.getenv('TAVILY_API_KEY', '').strip()

print(f"✓ .env file: {env_path}")
print(f"✓ File exists: {env_path.exists()}")
print(f"✓ OpenAI API Key loaded: {'Yes' if openai_key else 'No'}")
print(f"✓ Tavily API Key loaded: {'Yes' if tavily_key else 'No'}")

if not openai_key or not tavily_key:
    raise ValueError("Missing required credentials in .env")
print("✓ All credentials verified!")

## Load Environment & Verify Credentials

In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (.env location)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent  # notebooks -> healthbot -> project
src_path = project_root / 'healthbot' / 'src'

# Add src to path for module imports
sys.path.insert(0, str(src_path))
os.chdir(str(project_root))

print(f"✓ Project root: {project_root}")
print(f"✓ Src path added: {src_path}")
print(f"✓ Working directory: {os.getcwd()}")

## Setup: Environment & Path Configuration